<a href="https://www.kaggle.com/himanshu007121/merged-coswara-freq-data-analysis?scriptVersionId=87582770" target="_blank"><img align="left" alt="Kaggle" title="Open in Kaggle" src="https://kaggle.com/static/images/open-in-kaggle.svg"></a>

In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
#from scipy import fft
from scipy import signal as sig
from scipy.fft import fft, fftfreq
from IPython.display import Audio
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import math
import wave
import numpy as np
import matplotlib.pyplot as plt
import sys
import os
import librosa, librosa.display
import IPython.display as ipd
from pandas.plotting import register_matplotlib_converters
register_matplotlib_converters()
import scipy.io.wavfile as wavfile
import scipy
import scipy.fftpack
import numpy as np
from matplotlib import pyplot as plt
import seaborn as sns
import plotly.graph_objs as go
import warnings
warnings.simplefilter('ignore')

In [2]:
data = pd.read_csv('../input/freq-char-overall-dataset-creation/merged_data.csv')

In [3]:
data.head()

,Unnamed: 0,ID,Age (number),"Health status (e.g. : positive_mild, healthy,etc.)",Date when the user recorded and submitted the samples,Proficient in English (y/n),Gender (male/female/other),Country,Locality,State,...,"(820,840]","(840,860]","(860,880]","(880,900]","(900,920]","(920,940]","(940,960]","(960,980]","(980,1000]",source
0,0,9hXEs9OejdVxG6JJGCyKQpqVvy43,67,positive_moderate,20210719,y,male,India,Annur coimbator,Tamil Nadu,...,543695.2,615953.3,880054.9,770533.1,1.938898e+05,1.943286e+05,1.750968e+05,3.804597e+05,2.014790e+05,../input/correct-directory-extraction/cough_ou...
1,1,dPi1G2i821MDlbBUQWboTaExz023,24,healthy,20210816,y,male,India,Coimbatore,Tamil Nadu,...,5479105.0,5814686.0,5318021.0,5222335.0,2.284924e+06,2.251588e+06,2.259810e+06,2.213096e+06,2.142489e+06,../input/correct-directory-extraction/cough_ou...
2,2,TqZXC5KdCZOC4T7p69TUNnU9abD3,81,resp_illness_not_identified,20210816,n,male,India,Coimbatore,Tamil Nadu,...,2112739.0,1359745.0,2197208.0,1287216.0,2.597969e+06,1.661577e+06,1.435681e+06,1.535406e+06,1.558344e+06,../input/correct-directory-extraction/cough_ou...
3,3,356wSucV0OV6VQMvozjX63pIXrH2,24,no_resp_illness_exposed,20210728,y,male,India,Khagaria,Bihar,...,4808428.0,4258420.0,4594414.0,3564733.0,3.854638e+06,4.599038e+06,3.587551e+06,3.698612e+06,3.886427e+06,../input/correct-directory-extraction/cough_ou...
4,4,OPOiHCtB3WXhEdnsofhMpnP8bak1,38,recovered_full,20210715,y,male,India,Coimbatore,Tamil Nadu,...,1339581.0,789055.7,504429.1,272651.0,1.401376e+05,1.673620e+05,1.221615e+05,9.308663e+04,1.103769e+05,../input/correct-directory-extraction/cough_ou...


In [4]:
data = data.drop('Unnamed: 0', 1)

In [5]:
pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)
pd.set_option('display.max_colwidth', None)

In [6]:
data = data.rename(columns={'Health status (e.g. : positive_mild, healthy,etc.)': 'Status',
                            'Date when the user recorded and submitted the samples': 'Record_date',
                            'Gender (male/female/other)': 'gender',
                            'Smoker (True/False)':'Smoker',
                            'Cold  (True/False)':'Cold',
                            'Hypertension  (True/False)':'Hypertension',
                            'Diabetes  (True/False)':'Diabetes',
                            'Cough (True/False)':'Cough',
                            'Diarrheoa  (True/False)':'Diarrheoa',
                            'Fever (True/False)':'Fever',
                            'Status of COVID Test (p->Positive, n->Negative, na-> Not taken Test)':'COVID_status',
                            'Sore Throat (True/False)':'Sore_Throat',
                            'Chronic Lung Disease (True/False)':'Chronic_Lung_Disease',
                            'Pneumonia (True/False)':'Pneumonia',
                            'Age (number)':'Age'})

In [7]:
data.head()

,ID,Age,Status,Record_date,Proficient in English (y/n),gender,Country,Locality,State,Returning User (y/n),Smoker,Cold,Hypertension,Diabetes,Cough,Date of CT-Scan,CT-Scan (y/n if the user has taken a test),CT-Score,Diarrheoa,Fever,Loss of Smell & Taste (True/False),Muscle Pain (True/False),Type of test (RAT/RT-PCR),Date of COVID Test (if taken),COVID_status,Using Mask (y/n),"Vaccination status (y->both doses, p->one dose(partially vaccinated), n->no doses)",Breathing Difficulties (True/False),Respiratory illnesses other than the listed ones (True/False),Fatigue (True/False),Sore_Throat,Ischemic Heart Disease (True/False),Asthma (True/False),Pre-existing conditions other than the listed ones (True/False),Chronic_Lung_Disease,Pneumonia,"(0,20]","(20,40]","(40,60]","(60,80]","(80,100]","(100,120]","(120,140]","(140,160]","(160,180]","(180,200]","(200,220]","(220,240]","(240,260]","(260,280]","(280,300]","(300,320]","(320,340]","(340,360]","(360,380]","(380,400]","(400,420]","(420,440]","(440,460]","(460,480]","(480,500]","(500,520]","(520,540]","(540,560]","(560,580]","(580,600]","(600,620]","(620,640]","(640,660]","(660,680]","(680,700]","(700,720]","(720,740]","(740,760]","(760,780]","(780,800]","(800,820]","(820,840]","(840,860]","(860,880]","(880,900]","(900,920]","(920,940]","(940,960]","(960,980]","(980,1000]",source
0,9hXEs9OejdVxG6JJGCyKQpqVvy43,67,positive_moderate,20210719,y,male,India,Annur coimbator,Tamil Nadu,n,n,True,True,True,True,2021-07-17,y,6.0,True,True,True,True,rtpcr,2021-07-16,p,y,n,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,122462.20,110512.60,122038.70,165540.50,187686.50,234166.10,464990.6,1497269.0,4272285.0,6670527.0,6813176.0,7407627.0,5660090.0,7203625.0,10155080.0,8436202.0,5658043.0,2209144.0,2278392.0,1815391.0,787478.7,1155470.0,1920140.0,430031.7,381051.6,419583.0,1339097.0,1779969.0,3653937.0,3988582.0,2493865.0,2427367.0,1405778.0,580211.2,1024302.0,1169316.0,1471690.0,1479020.0,1526700.0,1655924.0,811980.9,543695.2,615953.3,880054.9,770533.1,1.938898e+05,1.943286e+05,1.750968e+05,3.804597e+05,2.014790e+05,../input/correct-directory-extraction/cough_outputs/9hXEs9OejdVxG6JJGCyKQpqVvy43/cough-shallow.wav
1,dPi1G2i821MDlbBUQWboTaExz023,24,healthy,20210816,y,male,India,Coimbatore,Tamil Nadu,n,n,NaN,NaN,NaN,NaN,NaN,n,NaN,NaN,NaN,NaN,NaN,rtpcr,2021-07-15,n,y,y,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,592186.50,765560.90,1176545.00,3394010.00,6056102.00,9178112.00,8054089.0,10818960.0,14923850.0,16215510.0,17957590.0,13504610.0,12587180.0,11845560.0,11671690.0,11583550.0,11329310.0,12382920.0,12156820.0,10602480.0,9186106.0,8328820.0,8118411.0,6559482.0,6876317.0,7129267.0,6856931.0,7466433.0,6866753.0,8820287.0,8991361.0,8519854.0,10095220.0,11077650.0,8958233.0,7698331.0,5565252.0,4931148.0,4414184.0,4509145.0,4584179.0,5479105.0,5814686.0,5318021.0,5222335.0,2.284924e+06,2.251588e+06,2.259810e+06,2.213096e+06,2.142489e+06,../input/correct-directory-extraction/cough_outputs/dPi1G2i821MDlbBUQWboTaExz023/cough-shallow.wav
2,TqZXC5KdCZOC4T7p69TUNnU9abD3,81,resp_illness_not_identified,20210816,n,male,India,Coimbatore,Tamil Nadu,n,y,True,NaN,NaN,True,NaN,n,NaN,NaN,NaN,NaN,NaN,NaN,NaN,na,y,y,True,True,NaN,NaN,NaN,NaN,NaN,NaN,NaN,316044.80,487795.00,638719.40,524027.10,368477.40,461292.90,640308.2,1208462.0,1969606.0,1678282.0,3848851.0,6622643.0,4935154.0,8735658.0,9117862.0,13349430.0,13153080.0,18491690.0,12085780.0,21799830.0,29205460.0,13011760.0,13641520.0,5651511.0,11057960.0,6876393.0,6519945.0,8594076.0,9566037.0,7159788.0,8367711.0,5400658.0,6378477.0,6994006.0,4164523.0,5966800.0,4388241.0,3485387.0,2947748.0,1288467.0,2685130.0,2112739.0,1359745.0,2197208.0,1287216.0,2.597969e+06,1.661577e+06,1.435681e+06,1.535406e+06,1.558344e+06,../input/correct-directory-extraction/cough_outputs/TqZXC5KdCZOC4T7p69TUNnU9abD3/cough-shallow.wav
3,356wSucV0OV6VQMvozjX63pIXrH2,24,no_resp_illness_exposed,20210728,y,male,India,Khagaria,Bihar,n,n,True,NaN,NaN,True,NaN,n,NaN,NaN,NaN,NaN,True,NaN,NaN,na,n,n,True,NaN,True,NaN,NaN,

In [8]:
cols = ['ID', 'Age', 'Status', 'Record_date',
       'gender', 'Country', 'Locality', 'State',
       'Smoker', 'Cold', 'Hypertension', 'Diabetes', 'Cough', 'Diarrheoa', 'Fever', 'Loss of Smell & Taste (True/False)',
       'Muscle Pain (True/False)', 'COVID_status', 'Using Mask (y/n)',
       'Vaccination status (y->both doses, p->one dose(partially vaccinated), n->no doses)',
       'Breathing Difficulties (True/False)',
       'Respiratory illnesses other than the listed ones (True/False)',
       'Fatigue (True/False)', 'Sore_Throat',
       'Ischemic Heart Disease (True/False)', 'Asthma (True/False)',
       'Pre-existing conditions other than the listed ones (True/False)',
       'Chronic_Lung_Disease', 'Pneumonia', '(0,20]', '(20,40]', '(40,60]',
       '(60,80]', '(80,100]', '(100,120]', '(120,140]', '(140,160]',
       '(160,180]', '(180,200]', '(200,220]', '(220,240]', '(240,260]',
       '(260,280]', '(280,300]', '(300,320]', '(320,340]', '(340,360]',
       '(360,380]', '(380,400]', '(400,420]', '(420,440]', '(440,460]',
       '(460,480]', '(480,500]', '(500,520]', '(520,540]', '(540,560]',
       '(560,580]', '(580,600]', '(600,620]', '(620,640]', '(640,660]',
       '(660,680]', '(680,700]', '(700,720]', '(720,740]', '(740,760]',
       '(760,780]', '(780,800]', '(800,820]', '(820,840]', '(840,860]',
       '(860,880]', '(880,900]', '(900,920]', '(920,940]', '(940,960]',
       '(960,980]', '(980,1000]', 'source']

freq_cols = ['(0,20]', '(20,40]', '(40,60]', '(60,80]', '(80,100]', '(100,120]',
       '(120,140]', '(140,160]', '(160,180]', '(180,200]', '(200,220]',
       '(220,240]', '(240,260]', '(260,280]', '(280,300]', '(300,320]',
       '(320,340]', '(340,360]', '(360,380]', '(380,400]', '(400,420]',
       '(420,440]', '(440,460]', '(460,480]', '(480,500]', '(500,520]',
       '(520,540]', '(540,560]', '(560,580]', '(580,600]', '(600,620]',
       '(620,640]', '(640,660]', '(660,680]', '(680,700]', '(700,720]',
       '(720,740]', '(740,760]', '(760,780]', '(780,800]', '(800,820]',
       '(820,840]', '(840,860]', '(860,880]', '(880,900]', '(900,920]',
       '(920,940]', '(940,960]', '(960,980]', '(980,1000]']

freq_cols_with_status_gender = ['(0,20]', '(20,40]', '(40,60]', '(60,80]', '(80,100]', '(100,120]',
       '(120,140]', '(140,160]', '(160,180]', '(180,200]', '(200,220]',
       '(220,240]', '(240,260]', '(260,280]', '(280,300]', '(300,320]',
       '(320,340]', '(340,360]', '(360,380]', '(380,400]', '(400,420]',
       '(420,440]', '(440,460]', '(460,480]', '(480,500]', '(500,520]',
       '(520,540]', '(540,560]', '(560,580]', '(580,600]', '(600,620]',
       '(620,640]', '(640,660]', '(660,680]', '(680,700]', '(700,720]',
       '(720,740]', '(740,760]', '(760,780]', '(780,800]', '(800,820]',
       '(820,840]', '(840,860]', '(860,880]', '(880,900]', '(900,920]',
       '(920,940]', '(940,960]', '(960,980]', '(980,1000]', 'COVID_status', 'gender']

demog_data = ['ID', 'Age', 'Status', 'Record_date', 'gender', 'Country', 'Locality',
       'State', 'Smoker', 'Cold', 'Hypertension', 'Diabetes', 'Cough',
       'Diarrheoa', 'Fever', 'Loss of Smell & Taste (True/False)',
       'Muscle Pain (True/False)', 'COVID_status', 'Using Mask (y/n)',
       'Vaccination status (y->both doses, p->one dose(partially vaccinated), n->no doses)',
       'Breathing Difficulties (True/False)',
       'Respiratory illnesses other than the listed ones (True/False)',
       'Fatigue (True/False)', 'Sore_Throat',
       'Ischemic Heart Disease (True/False)', 'Asthma (True/False)',
       'Pre-existing conditions other than the listed ones (True/False)',
       'Chronic_Lung_Disease', 'Pneumonia']

In [9]:
data = data[cols]

In [10]:
# #Preprocessing
options = ['male', 'female']
data = data[data['gender'].isin(options)] 
status = ['p', 'n']
data = data[data['COVID_status'].isin(status)] 
f_data = data

In [11]:
data['COVID_status'].value_counts()

p    470
n    201
Name: COVID_status, dtype: int64

In [12]:
import plotly.express as px
fig = px.box(data, y=freq_cols, color="COVID_status")
fig.show()

In [13]:
# fig = px.line(df3, x="Frequency", y="Amplitude", color='type')
# fig.update_layout(
#     title=("Frequency Vs Amplitude Distribution - <b>Overall<b>")
# )

In [14]:
# def plotting_function(data, column_name):
#     return 0

### Creating Lineplot Graphs

In [15]:
#COVID_Status + Gender
bigdata = data[freq_cols_with_status_gender]
data = bigdata.mean()
data_cov = bigdata[bigdata['COVID_status'] == 'p']
data_cov_male = data_cov[data_cov['gender'] == 'male']
data_cov_female = data_cov[data_cov['gender'] == 'female']
data_non_cov = bigdata[bigdata['COVID_status'] == 'n']
data_non_cov_male = data_non_cov[data_non_cov['gender'] == 'male']
data_non_cov_female = data_non_cov[data_non_cov['gender'] == 'female']

data_cov = data_cov.mean()
data_cov_male = data_cov_male.mean()
data_cov_female = data_cov_female.mean()
data_non_cov = data_non_cov.mean()
data_non_cov_male = data_non_cov_male.mean()
data_non_cov_female = data_non_cov_female.mean()

df = pd.DataFrame({'Frequency':data.index, 'Amplitude':data.values})
df_1 = pd.DataFrame({'Frequency':data_cov.index, 'Amplitude':data_cov.values})
df_2 = pd.DataFrame({'Frequency':data_non_cov.index, 'Amplitude':data_non_cov.values})
data_cov_male = pd.DataFrame({'Frequency':data_cov_male.index, 'Amplitude':data_cov_male.values})
data_cov_female = pd.DataFrame({'Frequency':data_cov_female.index, 'Amplitude':data_cov_female.values})
data_non_cov_male = pd.DataFrame({'Frequency':data_non_cov_male.index, 'Amplitude':data_non_cov_male.values})
data_non_cov_female = pd.DataFrame({'Frequency':data_non_cov_female.index, 'Amplitude':data_non_cov_female.values})

data_cov_male['COVID_status'] = 'p'
data_non_cov_male['COVID_status'] = 'n'
data_cov_female['COVID_status'] = 'p'
data_non_cov_female['COVID_status'] = 'n'

df_male = data_cov_male.append(data_non_cov_male, ignore_index=True)
df_female = data_cov_female.append(data_non_cov_female, ignore_index=True)

In [16]:
df_1['COVID_status'] = 'p'
df_2['COVID_status'] = 'n'

In [17]:
df_2.head()

,Frequency,Amplitude,COVID_status
0,"(0,20]",3.000976e+05,n
1,"(20,40]",3.211851e+05,n
2,"(40,60]",4.751229e+05,n
3,"(60,80]",7.559598e+05,n
4,"(80,100]",1.279357e+06,n


In [18]:
df3 = df_1.append(df_2, ignore_index=True)

In [19]:
import plotly.express as px

#df = px.data.gapminder().query("continent=='Oceania'")
fig = px.line(df3, x="Frequency", y="Amplitude", color='COVID_status')
fig.update_layout(
    title=("Frequency Vs Amplitude Distribution - <b>Overall<b>")
)

In [20]:
#Gender Wise Preprocessing
bigdata = f_data[demog_data]
#data = bigdata.mean()
data_cov = bigdata[bigdata['COVID_status'] == 'p']
data_cov_male = data_cov[data_cov['gender'] == 'male']
data_cov_female = data_cov[data_cov['gender'] == 'female']

In [21]:
bigdata.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 671 entries, 0 to 887
Data columns (total 29 columns):
 #   Column                                                                              Non-Null Count  Dtype 
---  ------                                                                              --------------  ----- 
 0   ID                                                                                  667 non-null    object
 1   Age                                                                                 671 non-null    int64 
 2   Status                                                                              671 non-null    object
 3   Record_date                                                                         671 non-null    int64 
 4   gender                                                                              671 non-null    object
 5   Country                                                                             671 non-null    object

In [22]:
fig = px.line(df_male, x="Frequency", y="Amplitude", color='COVID_status')
fig.update_layout(
    title=("Frequency Vs Amplitude Distribution - <b>Male Gender<b>")
)

In [23]:
# fig = px.line(data_cov_male, x="Age", color='COVID_status')
# fig.update_layout(
#     title=("Frequency Vs Amplitude Distribution - <b>Male Gender<b>")
# )
fig = px.histogram(data_cov_male, x="Age")
fig.update_layout(
    title=("Age Distribution - <b>Male Gender<b>")
)
fig.show()

In [24]:
data_cov_male.head()

,ID,Age,Status,Record_date,gender,Country,Locality,State,Smoker,Cold,Hypertension,Diabetes,Cough,Diarrheoa,Fever,Loss of Smell & Taste (True/False),Muscle Pain (True/False),COVID_status,Using Mask (y/n),"Vaccination status (y->both doses, p->one dose(partially vaccinated), n->no doses)",Breathing Difficulties (True/False),Respiratory illnesses other than the listed ones (True/False),Fatigue (True/False),Sore_Throat,Ischemic Heart Disease (True/False),Asthma (True/False),Pre-existing conditions other than the listed ones (True/False),Chronic_Lung_Disease,Pneumonia
0,9hXEs9OejdVxG6JJGCyKQpqVvy43,67,positive_moderate,20210719,male,India,Annur coimbator,Tamil Nadu,n,True,True,True,True,True,True,True,True,p,y,n,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,OPOiHCtB3WXhEdnsofhMpnP8bak1,38,recovered_full,20210715,male,India,Coimbatore,Tamil Nadu,n,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,p,n,p,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,Qcliznd3z1VdWmJOZh9nvlstTYv1,47,positive_moderate,20210717,male,India,Coimbatore,Tamil Nadu,n,True,NaN,True,True,True,True,True,True,p,y,n,NaN,NaN,True,NaN,NaN,NaN,NaN,NaN,NaN
9,dFtGnzYqh1NVAwQUQ3wkysqAe3n1,25,positive_asymp,20210729,male,India,NaN,Assam,n,NaN,NaN,NaN,True,NaN,NaN,NaN,NaN,p,n,y,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
10,rCsX9988lSeyFudpTucks6czFo32,42,positive_mild,20210717,male,India,Erode,Tamil Nadu,y,NaN,True,NaN,NaN,NaN,True,NaN,True,p,y,p,NaN,NaN,True,NaN,NaN,NaN,NaN,NaN,NaN


In [25]:
data_cov_male_india = data_cov_male[data_cov_male['Country'] == 'India']
#https://towardsdatascience.com/geographical-plotting-of-maps-with-plotly-4b5a5c95f02a

In [26]:
fig = px.line(df_female, x="Frequency", y="Amplitude", color='COVID_status')
fig.update_layout(
    title=("Frequency Vs Amplitude Distribution - <b>Female Gender<b>")
)

In [27]:
fig = px.histogram(data_cov_female, x="Age")
fig.update_layout(
    title=("Age Distribution - <b>Female Gender<b>")
)
fig.show()

In [28]:
f_data.to_csv('main_data.csv')